In [1]:
import regex as re
import pandas as pd
import os
import random
import numpy as np
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='your module name')

In [2]:
df = pd.read_csv('data.csv',index_col=0)
index = df.index
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import casual_tokenize
np.random.seed(42)

counter = CountVectorizer(tokenizer=casual_tokenize)
bow_docs = pd.DataFrame(counter.fit_transform(raw_documents=df.seg_text)
                        .toarray(), index=index)
column_nums, terms = zip(*sorted(zip(counter.vocabulary_.values(),
                                     counter.vocabulary_.keys())))
bow_docs.columns = terms
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import casual_tokenize
tfidf = TfidfVectorizer(tokenizer=casual_tokenize,)
tfidf_docs =tfidf.fit_transform(raw_documents=df.text).toarray()
tfidf_docs = tfidf_docs - tfidf_docs.mean(axis=0)


try:
    print('rumor0:'+df.loc['rumor0'].text)
    print(bow_docs.loc['rumor0'][bow_docs.loc['rumor0'] > 0].head())
except KeyError:
    print('rumor0!:'+df.loc['rumor0!'].text)
    print(bow_docs.loc['rumor0!'][bow_docs.loc['rumor0!'] > 0].head())

rumor0!:脑出血不能引起病人剧烈的头痛。
不能    1
剧烈    1
头痛    1
引起    1
病人    1
Name: rumor0!, dtype: int64


In [3]:
sentiments = df.sentiments
sentiment_dict = np.load('sentiment_dict.npy',allow_pickle=True).item()

In [5]:
from sklearn.decomposition import LatentDirichletAllocation as LDiA
ldia100 = LDiA(n_components=100, learning_method='batch')
ldia100 = ldia100.fit(bow_docs)
ldia_topic_vectors_100 = ldia100.transform(bow_docs)
columns100 = ['topic{}'.format(i) for i in range(ldia100.components_.shape[0])]
ldia_topic_vectors_100_doc = pd.DataFrame(ldia_topic_vectors_100,index=index, columns=columns100)
ldia_topic_vectors_100_sentiment = np.array([ldia_topic_vectors_100[i]*sentiments[i] for i in range(len(sentiments))])
ldia_topic_vectors_100_sentiment_doc = pd.DataFrame(ldia_topic_vectors_100_sentiment,index=index, columns=columns100)

ldia300 = LDiA(n_components=300, learning_method='batch')
ldia300 = ldia100.fit(bow_docs)
ldia_topic_vectors_300 = ldia300.transform(bow_docs)
columns300 = ['topic{}'.format(i) for i in range(ldia300.components_.shape[0])]
ldia_topic_vectors_300_doc = pd.DataFrame(ldia_topic_vectors_300,index=index, columns=columns300)
ldia_topic_vectors_300_sentiment = np.array([ldia_topic_vectors_300[i]*sentiments[i] for i in range(len(sentiments))])
ldia_topic_vectors_300_sentiment_doc = pd.DataFrame(ldia_topic_vectors_300_sentiment,index=index, columns=columns300)

In [6]:
from sklearn.decomposition import PCA
k=min(len(tfidf.vocabulary_),len(df))
k=100
pca = PCA(n_components=k)
pca_docs = pca.fit_transform(tfidf_docs)
base = sum(pca.singular_values_)
nums = pca.singular_values_.tolist()
pca = PCA(n_components=k)
pca_docs = pca.fit_transform(tfidf_docs)
pca_topic_vectors_100 = pca.transform(tfidf_docs)
columns = ['topic{}'.format(i) for i in range(pca_docs.shape[1])]
pca_topic_vectors_100_doc = pd.DataFrame(pca_topic_vectors_100, columns=columns, index=index)
pca_topic_vectors_100_sentiment = np.array([pca_topic_vectors_100[i]*sentiments[i] for i in range(len(sentiments))])
pca_topic_vectors_100_sentiment_doc = pd.DataFrame(pca_topic_vectors_100_sentiment, columns=columns, index=index)
k=300
pca = PCA(n_components=k)
pca_docs = pca.fit_transform(tfidf_docs)
base = sum(pca.singular_values_)
nums = pca.singular_values_.tolist()
pca = PCA(n_components=k)
pca_docs = pca.fit_transform(tfidf_docs)
pca_topic_vectors_300 = pca.transform(tfidf_docs)
columns = ['topic{}'.format(i) for i in range(pca_docs.shape[1])]
pca_topic_vectors_300_doc = pd.DataFrame(pca_topic_vectors_300, columns=columns, index=index)
pca_topic_vectors_300_sentiment = np.array([pca_topic_vectors_300[i]*sentiments[i] for i in range(len(sentiments))])
pca_topic_vectors_300_sentiment_doc = pd.DataFrame(pca_topic_vectors_300_sentiment, columns=columns, index=index)

In [7]:
tfidf.vocabulary_
#根据词项的频率对词汇表进行排序
#当对某个不按照最左边元素排序的序列解压并在排序后重新压缩时，可以使用zip(*sorted(zip(...)))
column_nums , terms = zip(*sorted(zip(tfidf.vocabulary_.values(), tfidf.vocabulary_.keys())))
weights = pd.DataFrame(pca.components_, columns=terms,
                       index = ['topic{}'.format(i) for i in range(pca.components_.shape[0])])
pd.options.display.max_columns = 8

In [8]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=100,n_iter=100)
svd_topic_vectors_100 = svd.fit_transform(tfidf_docs)
svd_topic_vectors_100 = (svd_topic_vectors_100.T / np.linalg.norm(svd_topic_vectors_100,axis=1)).T
svd_topic_vectors_100_doc = pd.DataFrame(svd_topic_vectors_100, columns=columns[:100], index=index)
svd_topic_vectors_100_sentiment = np.array([svd_topic_vectors_100[i]*sentiments[i] for i in range(len(sentiments))])
svd_topic_vectors_100_sentiment_doc = pd.DataFrame(svd_topic_vectors_100_sentiment, columns=columns[:100], index=index)

svd = TruncatedSVD(n_components=300,n_iter=100)
svd_topic_vectors_300 = svd.fit_transform(tfidf_docs)
svd_topic_vectors_300 = (svd_topic_vectors_300.T / np.linalg.norm(svd_topic_vectors_300,axis=1)).T
svd_topic_vectors_300_doc = pd.DataFrame(svd_topic_vectors_300, columns=columns[:300], index=index)
svd_topic_vectors_300_sentiment = np.array([svd_topic_vectors_300[i]*sentiments[i] for i in range(len(sentiments))])
svd_topic_vectors_300_sentiment_doc = pd.DataFrame(svd_topic_vectors_300_sentiment, columns=columns[:300], index=index)

In [9]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split

def lda_predic (name,vector,df,over_sampling=False):
    X_train,X_test,y_train,y_test = train_test_split(vector,df.rumor,test_size=0.2 ,random_state=42)
    if over_sampling:
        smote = SMOTE(random_state=42)
        # 进行过采样
        X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
        print("过采样前训练集中类别0的数量：", sum(y_train==0))
        print("过采样前训练集中类别1的数量：", sum(y_train==1))

        print("过采样后训练集中类别0的数量：", sum(y_train_res==0))
        print("过采样后训练集中类别1的数量：", sum(y_train_res==1))

    lda = LDA(n_components=1)
    lda = lda.fit(X_train_res, y_train_res)
    df[name] = lda.predict(vector)
    round(float(lda.score(X_test, y_test)), 3)
    from sklearn import metrics
    y_true = df.loc[y_test.index].rumor
    y_pred = df.loc[y_test.index][name]
    loss_total = 0
    acc = metrics.accuracy_score(y_true, y_pred)
    report = metrics.classification_report(y_true, y_pred, target_names=["non-rumor","rumor"], digits=4)
    confusion = metrics.confusion_matrix(y_true, y_pred)
    print(report)
    with open("report/"+name+'.txt', 'w', encoding='utf-8') as f:
        print(report,file=f)
    print(confusion)
    with open("report_matrix/"+name+'_matrix'+'.txt', 'w', encoding='utf-8') as f:
        print(confusion,file=f)
    np.savez("report_matrix/"+name+'_matrix'+'.npz', matrix= confusion)

In [10]:
lda_predic('tfidf',tfidf_docs,df,over_sampling=True)

过采样前训练集中类别0的数量： 1502
过采样前训练集中类别1的数量： 725
过采样后训练集中类别0的数量： 1502
过采样后训练集中类别1的数量： 1502
              precision    recall  f1-score   support

   non-rumor     0.9388    0.4986    0.6513       369
       rumor     0.4875    0.9362    0.6412       188

    accuracy                         0.6463       557
   macro avg     0.7132    0.7174    0.6462       557
weighted avg     0.7865    0.6463    0.6479       557

[[184 185]
 [ 12 176]]


In [11]:
lda_predic('pca_100',pca_topic_vectors_100_doc,df,over_sampling=True)

过采样前训练集中类别0的数量： 1502
过采样前训练集中类别1的数量： 725
过采样后训练集中类别0的数量： 1502
过采样后训练集中类别1的数量： 1502
              precision    recall  f1-score   support

   non-rumor     0.9522    0.7019    0.8081       369
       rumor     0.6140    0.9309    0.7400       188

    accuracy                         0.7792       557
   macro avg     0.7831    0.8164    0.7740       557
weighted avg     0.8381    0.7792    0.7851       557

[[259 110]
 [ 13 175]]


In [12]:
lda_predic('pca_300',pca_topic_vectors_300_doc,df,over_sampling=True)

过采样前训练集中类别0的数量： 1502
过采样前训练集中类别1的数量： 725
过采样后训练集中类别0的数量： 1502
过采样后训练集中类别1的数量： 1502
              precision    recall  f1-score   support

   non-rumor     0.9483    0.7453    0.8346       369
       rumor     0.6479    0.9202    0.7604       188

    accuracy                         0.8043       557
   macro avg     0.7981    0.8327    0.7975       557
weighted avg     0.8469    0.8043    0.8096       557

[[275  94]
 [ 15 173]]


In [13]:
lda_predic('pca_100_sentiment',pca_topic_vectors_100_sentiment_doc,df,over_sampling=True)

过采样前训练集中类别0的数量： 1502
过采样前训练集中类别1的数量： 725
过采样后训练集中类别0的数量： 1502
过采样后训练集中类别1的数量： 1502
              precision    recall  f1-score   support

   non-rumor     0.9027    0.7290    0.8066       369
       rumor     0.6139    0.8457    0.7114       188

    accuracy                         0.7684       557
   macro avg     0.7583    0.7874    0.7590       557
weighted avg     0.8052    0.7684    0.7745       557

[[269 100]
 [ 29 159]]


In [14]:
lda_predic('pca_300_sentiment',pca_topic_vectors_300_sentiment_doc,df,over_sampling=True)

过采样前训练集中类别0的数量： 1502
过采样前训练集中类别1的数量： 725
过采样后训练集中类别0的数量： 1502
过采样后训练集中类别1的数量： 1502
              precision    recall  f1-score   support

   non-rumor     0.9133    0.7425    0.8191       369
       rumor     0.6304    0.8617    0.7281       188

    accuracy                         0.7828       557
   macro avg     0.7718    0.8021    0.7736       557
weighted avg     0.8178    0.7828    0.7884       557

[[274  95]
 [ 26 162]]


In [15]:
lda_predic('svd_100',svd_topic_vectors_100_doc,df,over_sampling=True)

过采样前训练集中类别0的数量： 1502
过采样前训练集中类别1的数量： 725
过采样后训练集中类别0的数量： 1502
过采样后训练集中类别1的数量： 1502
              precision    recall  f1-score   support

   non-rumor     0.8635    0.6856    0.7644       369
       rumor     0.5606    0.7872    0.6549       188

    accuracy                         0.7199       557
   macro avg     0.7120    0.7364    0.7096       557
weighted avg     0.7613    0.7199    0.7274       557

[[253 116]
 [ 40 148]]


In [16]:
lda_predic('svd_300',svd_topic_vectors_300_doc,df,over_sampling=True)

过采样前训练集中类别0的数量： 1502
过采样前训练集中类别1的数量： 725
过采样后训练集中类别0的数量： 1502
过采样后训练集中类别1的数量： 1502
              precision    recall  f1-score   support

   non-rumor     0.9357    0.7100    0.8074       369
       rumor     0.6137    0.9043    0.7312       188

    accuracy                         0.7756       557
   macro avg     0.7747    0.8071    0.7693       557
weighted avg     0.8270    0.7756    0.7817       557

[[262 107]
 [ 18 170]]


In [17]:
lda_predic('svd_100_sentiment',svd_topic_vectors_100_sentiment_doc,df,over_sampling=True)

过采样前训练集中类别0的数量： 1502
过采样前训练集中类别1的数量： 725
过采样后训练集中类别0的数量： 1502
过采样后训练集中类别1的数量： 1502
              precision    recall  f1-score   support

   non-rumor     0.8653    0.6965    0.7718       369
       rumor     0.5692    0.7872    0.6607       188

    accuracy                         0.7271       557
   macro avg     0.7173    0.7419    0.7162       557
weighted avg     0.7654    0.7271    0.7343       557

[[257 112]
 [ 40 148]]


In [18]:
lda_predic('svd_300_sentiment',svd_topic_vectors_300_sentiment_doc,df,over_sampling=True)

过采样前训练集中类别0的数量： 1502
过采样前训练集中类别1的数量： 725
过采样后训练集中类别0的数量： 1502
过采样后训练集中类别1的数量： 1502
              precision    recall  f1-score   support

   non-rumor     0.9352    0.6260    0.7500       369
       rumor     0.5548    0.9149    0.6908       188

    accuracy                         0.7235       557
   macro avg     0.7450    0.7705    0.7204       557
weighted avg     0.8068    0.7235    0.7300       557

[[231 138]
 [ 16 172]]


In [19]:
lda_predic('ldia_100',ldia_topic_vectors_100_doc,df,over_sampling=True)

过采样前训练集中类别0的数量： 1502
过采样前训练集中类别1的数量： 725
过采样后训练集中类别0的数量： 1502
过采样后训练集中类别1的数量： 1502
              precision    recall  f1-score   support

   non-rumor     0.7483    0.5962    0.6637       369
       rumor     0.4335    0.6064    0.5055       188

    accuracy                         0.5996       557
   macro avg     0.5909    0.6013    0.5846       557
weighted avg     0.6420    0.5996    0.6103       557

[[220 149]
 [ 74 114]]


In [20]:
lda_predic('ldia_300',ldia_topic_vectors_300_doc,df,over_sampling=True)

过采样前训练集中类别0的数量： 1502
过采样前训练集中类别1的数量： 725
过采样后训练集中类别0的数量： 1502
过采样后训练集中类别1的数量： 1502
              precision    recall  f1-score   support

   non-rumor     0.7509    0.5881    0.6596       369
       rumor     0.4328    0.6170    0.5088       188

    accuracy                         0.5978       557
   macro avg     0.5919    0.6025    0.5842       557
weighted avg     0.6435    0.5978    0.6087       557

[[217 152]
 [ 72 116]]


In [21]:
lda_predic('ldia_100_sentiment',ldia_topic_vectors_100_sentiment_doc,df,over_sampling=True)

过采样前训练集中类别0的数量： 1502
过采样前训练集中类别1的数量： 725
过采样后训练集中类别0的数量： 1502
过采样后训练集中类别1的数量： 1502
              precision    recall  f1-score   support

   non-rumor     0.7754    0.5989    0.6758       369
       rumor     0.4559    0.6596    0.5391       188

    accuracy                         0.6194       557
   macro avg     0.6157    0.6292    0.6075       557
weighted avg     0.6676    0.6194    0.6297       557

[[221 148]
 [ 64 124]]


In [22]:
lda_predic('ldia_300_sentiment',ldia_topic_vectors_300_sentiment_doc,df,over_sampling=True)

过采样前训练集中类别0的数量： 1502
过采样前训练集中类别1的数量： 725
过采样后训练集中类别0的数量： 1502
过采样后训练集中类别1的数量： 1502
              precision    recall  f1-score   support

   non-rumor     0.7345    0.5474    0.6273       369
       rumor     0.4078    0.6117    0.4894       188

    accuracy                         0.5691       557
   macro avg     0.5712    0.5796    0.5583       557
weighted avg     0.6243    0.5691    0.5808       557

[[202 167]
 [ 73 115]]
